In [1]:
import os
from dotenv import load_dotenv

from langchain.evaluation.qa.eval_chain import QAEvalChain
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
enhanced_graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [7]:
def evaluate_graph_rag(graph_chain, examples):

    # Generate predictions by querying the graph
    predictions = []
    for ex in examples:
        graph_response = graph_chain.invoke({"query": ex["query"]})
        predictions.append({"result": graph_response["result"].strip()})

    # Run evaluation
    eval_chain = QAEvalChain.from_llm(llm)
    results = eval_chain.evaluate(examples, predictions)

    # Print output
    correct = 0
    for i, res in enumerate(results):
        print(f"Query: {examples[i]['query']}")
        print(f"Prediction from graph: {predictions[i]['result']}")
        print(f"Gold answer: {examples[i]['answer']}")
        print(f"Grade: {res['results']}")
        print("---")
        if res["results"] == "CORRECT":
            correct += 1

    accuracy = correct / len(examples)
    print(f"Graph QA Accuracy: {accuracy:.2f}")

In [4]:
examples = [
    {"query": "What training method does the International Ski and Snowboard Federation use?",
     "answer": "Grass skiing"},
    {"query": "How does research play a role?",
     "answer": "Research improves competition by looking at Surface Science and Friction Reduction and has implications for Competitive Performance and Recreational Enjoyment"},
    {"query": "What country was the men's overall winner of the 2024–25 Fis Alpine Ski World Cup representing?",
     "answer": "Switzerland"},
]

In [11]:
CYPHER_GENERATION_TEMPLATE = """

You are an expert Neo4j developer. Your job is to generate a Cypher query that 
will traverse exactly 1–3 hops out from your focus node.

Steps:

1. Identify the main entity in the question and Capitalize Multi-Word IDs
2. Write a variable-length MATCH using the new syntax:
   MATCH (a {{id: "{{entity}}"}})-[r]-{{1,3}}(b)
   RETURN a, r, b;

You MUST use {{1,3}} for setting the number of hops instead of [r*0..3].
Do NOT use WHERE length(r) >= 1 AND length(r) <= 3 to specify it either.

Schema:
{schema}

Question:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

enhanced_graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [12]:
evaluate_graph_rag(enhanced_graph_chain, examples)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a {id: "International Ski And Snowboard Federation"})-[r]-{1,3}(b)
RETURN a, r, b;

Full Context:
[{'a': {'id': 'International Ski And Snowboard Federation'}, 'r': [({'id': 'International Ski And Snowboard Federation'}, 'RECOGNIZED_SKI_FLYING', {'id': '1938'})], 'b': {'id': '1938'}}, {'a': {'id': 'International Ski And Snowboard Federation'}, 'r': [({'id': 'International Ski And Snowboard Federation'}, 'RECOGNIZED_SKI_FLYING', {'id': '1938'}), ({'id': '1938'}, 'BEFORE', {'id': 'World War Ii'})], 'b': {'id': 'World War Ii'}}, {'a': {'id': 'International Ski And Snowboard Federation'}, 'r': [({'id': 'International Ski And Snowboard Federation'}, 'RECOGNIZED_SKI_FLYING', {'id': '1938'}), ({'id': '1938'}, 'BEFORE', {'id': 'World War Ii'}), ({'summary': 'The International Ski and Snowboard Federation, also known as FIS (French: Fédération Internationale de Ski et de Snowboard), is the highest international governin